In [1]:
import spacy
import pickle

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
import tensorflow as tf
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

In [30]:
def process_data(input_data):
    """Extract the first file enclosed in a zip file as a list of words."""
    data = ''
    with zipfile.ZipFile(input_data) as f:
        for i in f.namelist():
            data += tf.compat.as_str(f.read(i)).strip()
    nlp = spacy.load('en')
    paredDate = nlp(data)
    #TODO: do we need lemma??
    ret = []
    for tok in paredDate:
        if tok.lemma_ and not (tok.is_punct or tok.is_space):
            temp = tok.lower_
            if temp[-1] == '%':
                temp = 'PERCENT'
            elif tok.like_num:
                temp = 'NUMBER'
            ret.append(temp)
    file_name = 'tokenize.pickle'
    file = open(file_name, 'wb')
    pickle.dump(ret, file)
    return file_name

In [31]:
file_name = process_data('BBC_Data.zip')
with open(file_name, 'rb') as file:
    data = pickle.load(file)

In [40]:
prob_dict = {}
def get_probabilty(file_name):
    import math
    with open(file_name, 'rb') as file:
        data = pickle.load(file)
    for i in data:
        if i in prob_dict:
            prob_dict[i] += 1
        else:
            prob_dict[i] = 1
    length = len(data)
    print(length)
    def google_prob(i):
        freq = i / length * 100
        return min( 1, (math.sqrt(freq) + 1) / freq)
    
    for i in prob_dict:
        prob_dict[i] = google_prob(prob_dict[i])

In [41]:
get_probabilty('tokenize.pickle')
print(prob_dict)

874807
{'qantas': 1, 'sees': 1, 'profits': 1, 'fly': 1, 'to': 0.9389175128037511, 'record': 1, 'australian': 1, 'airline': 1, 'has': 1, 'posted': 1, 'a': 1, 'fiscal': 1, 'first': 1, 'half': 1, 'profit': 1, 'thanks': 1, 'cost': 1, 'cutting': 1, 'measures': 1, 'net': 1, 'in': 1, 'the': 0.574036691479228, 'NUMBER': 1, 'months': 1, 'ending': 1, 'december': 1, 'rose': 1, 'PERCENT': 1, 'a$': 1, 'm': 1, '$': 1, '£': 1, 'from': 1, 'year': 1, 'earlier': 1, 'analysts': 1, 'expected': 1, 'figure': 1, 'closer': 1, 'shares': 1, 'fell': 1, 'almost': 1, 'however': 1, 'after': 1, 'it': 1, 'warned': 1, 'that': 1, 'earnings': 1, 'growth': 1, 'would': 1, 'slow': 1, 'second': 1, 'sales': 1, 'will': 1, 'dip': 1, 'by': 1, 'at': 1, 'least': 1, 'indian': 1, 'ocean': 1, 'tsunami': 1, 'devastated': 1, 'many': 1, 'holiday': 1, 'destinations': 1, 'said': 1, 'affected': 1, 'travel': 1, 'patterns': 1, 'ways': 1, 'we': 1, 'were': 1, 'bit': 1, 'surprised': 1, 'about': 1, 'chief': 1, 'executive': 1, 'geoff': 1, 'dixon

In [17]:
vocabulary_size = 50000 # This variable is used to define the maximum vocabulary size.

def build_dataset(words, n_words):
    """Process raw inputs into a dataset. 
       words: a list of words, i.e., the input data
       n_words: Vocab_size to limit the size of the vocabulary. Other words will be mapped to 'UNK'
    """
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # i.e., one of the 'UNK' words
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(data, vocabulary_size)

In [27]:
print(count[:10])

[['UNK', 0], (1, 8253), (2, 7456), (3, 2995), (4, 2968), (5, 2042), (6, 1991), (7, 1913), (8, 1813), (9, 1534)]


In [28]:
test = 'more, most, good'
nlp = spacy.load('en')
paredDate = nlp(test)
for i in paredDate:
    print(i.lemma_)

more
,
most
,
good


In [29]:
data_index = 0 
# the variable is abused in this implementation. 
# Outside the sample generation loop, it is the position of the sliding window: from data_index to data_index + span
# Inside the sample generation loop, it is the next word to be added to a size-limited buffer. 

def generate_batch(batch_size, num_samples, skip_window):
    global data_index   
    
    assert batch_size % num_samples == 0
    assert num_samples <= 2 * skip_window
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # span is the width of the sliding window
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span]) # initial buffer content = first sliding window
    
    print('data_index = {}, buffer = {}'.format(data_index, [reverse_dictionary[w] for w in buffer]))

    data_index += span
    
    for i in range(batch_size // num_samples):
        context_words = [w for w in range(span) if w != skip_window]
        random.shuffle(context_words)
        words_to_use = collections.deque(context_words) # now we obtain a random list of context words
        for j in range(num_samples): # generate the training pairs
            batch[i * num_samples + j] = buffer[skip_window]
            context_word = words_to_use.pop()
            labels[i * num_samples + j, 0] = buffer[context_word] # buffer[context_word] is a random context word
        
        # slide the window to the next position    
        if data_index == len(data):
            buffer = data[:span]
            data_index = span
        else: 
            buffer.append(data[data_index]) # note that due to the size limit, the left most word is automatically removed from the buffer.
            data_index += 1
        
        print('data_index = {}, buffer = {}'.format(data_index, [reverse_dictionary[w] for w in buffer]))
        
    # end-of-for
    data_index = (data_index + len(data) - span) % len(data) # move data_index back by `span`
    return batch, labels

# 
print('data[0:10] = {}'.format([reverse_dictionary[i] for i in data[:10]]))

print('\n.. First batch')
batch, labels = generate_batch(batch_size=8, num_samples=2, skip_window=1)
for i in range(8):
    print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])
print(data_index)
    
print('\n.. Second batch')
batch, labels = generate_batch(batch_size=8, num_samples=2, skip_window=1)
for i in range(8):
    print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])
print(data_index)

data[0:10] = [3075, 71, 280, 892, 88, 625, 683, 3075, 589, 88]

.. First batch
data_index = 0, buffer = [3075, 71, 280]
data_index = 4, buffer = [71, 280, 892]
data_index = 5, buffer = [280, 892, 88]
data_index = 6, buffer = [892, 88, 625]
data_index = 7, buffer = [88, 625, 683]
71 -> 280
71 -> 3075
280 -> 71
280 -> 892
892 -> 88
892 -> 280
88 -> 892
88 -> 625
4

.. Second batch
data_index = 4, buffer = [88, 625, 683]
data_index = 8, buffer = [625, 683, 3075]
data_index = 9, buffer = [683, 3075, 589]
data_index = 10, buffer = [3075, 589, 88]
data_index = 11, buffer = [589, 88, 1797]
625 -> 88
625 -> 683
683 -> 625
683 -> 3075
3075 -> 683
3075 -> 589
589 -> 88
589 -> 3075
8


In [30]:
# Specification of Training data:
batch_size = 128      # Size of mini-batch for skip-gram model.
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right of the target word.
num_samples = 2         # How many times to reuse an input to generate a label.
num_sampled = 64      # Sample size for negative examples.
logs_path = './log/'

# Specification of test Sample:
sample_size = 20       # Random sample of words to evaluate similarity.
sample_window = 100    # Only pick samples in the head of the distribution.
sample_examples = np.random.choice(sample_window, sample_size, replace=False) # Randomly pick a sample of size 16

## Constructing the graph...
graph = tf.Graph()

with graph.as_default():
    
    with tf.device('/cpu:0'):
        # Placeholders to read input data.
        with tf.name_scope('Inputs'):
            train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
            train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
            
        # Look up embeddings for inputs.
        with tf.name_scope('Embeddings'):            
            sample_dataset = tf.constant(sample_examples, dtype=tf.int32)
            embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
            
            # Construct the variables for the NCE loss
            nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                                                      stddev=1.0 / math.sqrt(embedding_size)))
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        with tf.name_scope('Loss'):
            loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights, biases=nce_biases, 
                                             labels=train_labels, inputs=embed, 
                                             num_sampled=num_sampled, num_classes=vocabulary_size))
        
        # Construct the Gradient Descent optimizer using a learning rate of 0.01.
        with tf.name_scope('Gradient_Descent'):
            optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1).minimize(loss)

        # Normalize the embeddings to avoid overfitting.
        with tf.name_scope('Normalization'):
            norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
            normalized_embeddings = embeddings / norm
            
        sample_embeddings = tf.nn.embedding_lookup(normalized_embeddings, sample_dataset)
        similarity = tf.matmul(sample_embeddings, normalized_embeddings, transpose_b=True)
        
        # Add variable initializer.
        init = tf.global_variables_initializer()
        
        
        # Create a summary to monitor cost tensor
        tf.summary.scalar("cost", loss)
        # Merge all summary variables.
        merged_summary_op = tf.summary.merge_all()

In [31]:
num_steps = 130001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    session.run(init)
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    print('Initializing the model')
    
    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_samples, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        
        # We perform one update step by evaluating the optimizer op using session.run()
        _, loss_val, summary = session.run([optimizer, loss, merged_summary_op], feed_dict=feed_dict)
        
        summary_writer.add_summary(summary, step )
        average_loss += loss_val

        if step % 5000 == 0:
            if step > 0:
                average_loss /= 5000
            
                # The average loss is an estimate of the loss over the last 5000 batches.
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0

        # Evaluate similarity after every 10000 iterations.
        if step % 10000 == 0:
            sim = similarity.eval() #
            for i in range(sample_size):
                sample_word = reverse_dictionary[sample_examples[i]]
                top_k = 10  # Look for top-10 neighbours for words in sample set.
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % sample_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
            print()
            
    final_embeddings = normalized_embeddings.eval()

Initializing the model
data_index = 8, buffer = [589, 88, 1797]
data_index = 12, buffer = [88, 1797, 124]
data_index = 13, buffer = [1797, 124, 280]
data_index = 14, buffer = [124, 280, 982]
data_index = 15, buffer = [280, 982, 104]
data_index = 16, buffer = [982, 104, 120]
data_index = 17, buffer = [104, 120, 618]
data_index = 18, buffer = [120, 618, 117]
data_index = 19, buffer = [618, 117, 280]
data_index = 20, buffer = [117, 280, 33]
data_index = 21, buffer = [280, 33, 67]
data_index = 22, buffer = [33, 67, 2032]
data_index = 23, buffer = [67, 2032, 260]
data_index = 24, buffer = [2032, 260, 63]
data_index = 25, buffer = [260, 63, 7824]
data_index = 26, buffer = [63, 7824, 2902]
data_index = 27, buffer = [7824, 2902, 16067]
data_index = 28, buffer = [2902, 16067, 10]
data_index = 29, buffer = [16067, 10, 15]
data_index = 30, buffer = [10, 15, 16068]
data_index = 31, buffer = [15, 16068, 10]
data_index = 32, buffer = [16068, 10, 13]
data_index = 33, buffer = [10, 13, 12182]
data_ind

KeyError: 27620